# Feature Crosses

## Setup

In [0]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
from sklearn import metrics
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [0]:
california_housing_dataframe = pd.read_csv("https://storage.googleapis.com/mledu-datasets/california_housing_train.csv", sep=",")
california_housing_dataframe = california_housing_dataframe.reindex(np.random.permutation(california_housing_dataframe.index))

## Preprocess

In [0]:
def preprocess_features(california_housing_dataframe):
  """Prepares input features from California housing data set.
  
  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data from the California housing data set.
  Returns:
    A DataFrame that contains the features to be used for the model, including synthetic features.
  """
  
  selected_features = california_housing_dataframe[[
    "latitude",
    "longitude",
    "housing_median_age",
    "total_rooms",
    "total_bedrooms",
    "population",
    "households",
    "median_income"
  ]]
  
  processed_features = selected_features.copy()
  
  # Create a synthetic feature.
  processed_features["rooms_per_person"] = (california_housing_dataframe["total_rooms"] / california_housing_dataframe["population"])
  
  return processed_features

In [0]:
def preprocess_targets(california_housing_dataframe):
  """Prepares target features (i.e., labels) from California housing data set.
  
  Args:
    california_housing_dataframe: A Pandas DataFrame expected to contain data from the California housing data set.
  Returns:
    A DataFrame that contains the target feature.
  """
  
  output_targets = pd.DataFrame()
  
  # Scale the target to be in units of thousands of dollars.
  output_targets["median_house_value"] = (california_housing_dataframe["median_house_value"] / 1000.0)
  
  return output_targets

In [0]:
# Choose the first 12000 (out of 17000) examples for training.
training_examples = preprocess_features(california_housing_dataframe.head(12000))
training_targets = preprocess_targets(california_housing_dataframe.head(12000))

# Choose the last 5000 (out of 17000) examples for validation.
validation_examples = preprocess_features(california_housing_dataframe.tail(5000))
validation_targets = preprocess_targets(california_housing_dataframe.tail(5000))

# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.describe())
print("Validation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Validation targets summary:")
display.display(validation_targets.describe())

## Features

In [0]:
def get_quantile_based_boundaries(feature_values, num_buckets):
  boundaries = np.arange(1.0, num_buckets) / num_buckets
  quantiles = feature_values.quantile(boundaries)
  
  return [quantiles[q] for q in quantiles.keys()]

In [0]:
def construct_feature_columns():
  """Construct the TensorFlow Feature Columns.
  
  Returns:
    A set of feature columns.
  """
  
  households = tf.feature_column.numeric_column("households")
  longitude = tf.feature_column.numeric_column("longitude")
  latitude = tf.feature_column.numeric_column("latitude")
  housing_median_age = tf.feature_column.numeric_column("housing_median_age")
  median_income = tf.feature_column.numeric_column("median_income")
  rooms_per_person = tf.feature_column.numeric_column("rooms_per_person")
  
  # Divide households into 7 buckets.
  bucketized_households = tf.feature_column.bucketized_column(households, boundaries=get_quantile_based_boundaries(training_examples["households"], 7))
  
  # Divide longitude into 10 buckets.
  bucketized_longitude = tf.feature_column.bucketized_column(longitude, boundaries=get_quantile_based_boundaries(training_examples["longitude"], 10))
  
  # Divide latitude into 10 buckets.
  bucketized_latitude = tf.feature_column.bucketized_column(latitude, boundaries=get_quantile_based_boundaries(training_examples["latitude"], 10))
  
  # Divide housing_median_age into 7 buckets.
  bucketized_housing_median_age = tf.feature_column.bucketized_column(housing_median_age, boundaries=get_quantile_based_boundaries(training_examples["housing_median_age"], 7))
  
  # Divide median_income into 7 buckets.
  bucketized_median_income = tf.feature_column.bucketized_column(median_income, boundaries=get_quantile_based_boundaries(training_examples["median_income"], 7))
  
  # Divide rooms_per_person into 7 buckets.
  bucketized_rooms_per_person = tf.feature_column.bucketized_column(rooms_per_person, boundaries=get_quantile_based_boundaries(training_examples["rooms_per_person"], 7))
  
  # Longitude x latitude feature cross
  long_x_lat = tf.feature_column.crossed_column(set([bucketized_longitude, bucketized_latitude]), hash_bucket_size=1000)
  
  feature_columns = set([
    bucketized_longitude,
    bucketized_latitude,
    bucketized_housing_median_age,
    bucketized_households,
    bucketized_median_income,
    bucketized_rooms_per_person,
    long_x_lat
  ])
  
  return feature_columns

## Train

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
  """Trains a linear regression model of multiple features.
  
  Args:
    features: pandas DataFrame of features.
    targets: pandas DataFrame of targets.
    batch_size: Size of batches to be passed to the model.
    shuffle: True or False. Whether to shuffle the data.
    num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely.
  Returns:
    Tuple of (features, labels) for next data batch.
  """
  
  # Convert pandas data into a dict of np arrays.
  features = {key: np.array(value) for key, value in dict(features).items()}
  
  # Construct a dataset, and configure batching/repeating.
  ds = Dataset.from_tensor_slices((features, targets))  # warning: 2GB limit
  ds = ds.batch(batch_size).repeat(num_epochs)
  
  # Shuffle the data, if specified.
  if shuffle:
    ds = ds.shuffle(10000)
  
  # Return the next batch of data.
  features, labels = ds.make_one_shot_iterator().get_next()
  
  return features, labels

In [0]:
def train_model(learning_rate, steps, batch_size, feature_columns, training_examples, training_targets, validation_examples, validation_targets):
  """Trains a linear regression model of multiple features.
     In addition to training, this function also prints training progress information,
     as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    feature_columns: A `set` specifying the input feature columns to use.
    training_examples: A `DataFrame` containing one or more columns from `california_housing_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from `california_housing_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from `california_housing_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from `california_housing_dataframe` to use as target for validation.
  Returns:
    A `LinearRegressor` object trained on the training data.
  """
  
  periods = 10
  steps_per_period = steps / periods
  
  # Create a linear regressor object.
  my_optimizer = tf.train.FtrlOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  
  linear_regressor = tf.estimator.LinearRegressor(feature_columns=feature_columns, optimizer=my_optimizer)
  
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, training_targets["median_house_value"], batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, training_targets["median_house_value"], num_epochs=1, shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, validation_targets["median_house_value"], num_epochs=1, shuffle=False)
  
  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  
  for period in range(0, periods):
    # Train the model, starting from the prior state.
    linear_regressor.train(input_fn=training_input_fn, steps=steps_per_period)
    
    # Take a break and compute predictions.
    training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = linear_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(metrics.mean_squared_error(validation_predictions, validation_targets))
    
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
    
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
    
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  return linear_regressor

In [0]:
train_model(
  learning_rate=1.0,
  steps=500,
  batch_size=100,
  feature_columns=construct_feature_columns(),
  training_examples=training_examples,
  training_targets=training_targets,
  validation_examples=validation_examples,
  validation_targets=validation_targets)